In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

     |████████████████████████████████| 720 kB 7.6 MB/s 
     |████████████████████████████████| 1.2 MB 44.3 MB/s 
     |████████████████████████████████| 186 kB 46.3 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 51 kB 342 kB/s 
Mounted at /content/gdrive


In [2]:
from fastai.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

In [3]:
Path.BASE_PATH = path

In [4]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

### Data Preprocessing
- Preprocess the dataset for the language model
- Manually tokenize and numericalize the data
- Create batches for the model to read

In [5]:
text = " . ".join(line.strip() for line in lines)
text[:50]

'one . two . three . four . five . six . seven . ei'

In [6]:
# get the tokens by splitting on spaces
tokens = text.split()
tokens[:10]

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

In [7]:
vocab = list(set(tokens))
vocab[:10]

['two',
 'seventeen',
 'hundred',
 'eight',
 'three',
 'thirteen',
 'six',
 'four',
 'eighteen',
 '.']

In [8]:
word2index = {word: i for i, word in enumerate(vocab)}
word2index['two']

0

In [10]:
num_tokens = [word2index[word] for word in tokens]
num_tokens[:10]

[27, 9, 0, 9, 4, 9, 7, 9, 19, 9]

In [13]:
len_tks = 6

for i in range(0,len_tks-4,3):
  print(i)

0


In [12]:
print(len(tokens))

63095


In [44]:
# want sequences of consecutive characters
seq_len = 16
seqs = [(torch.Tensor(num_tokens[i: i+seq_len]).int(), torch.Tensor(num_tokens[i+1:i+seq_len+1]).int()) for i in range(0,len(num_tokens)-seq_len-1, seq_len)]
seqs[:2]                                                                                         

[(tensor([27,  9,  0,  9,  4,  9,  7,  9, 19,  9,  6,  9, 16,  9,  3,  9], dtype=torch.int32),
  tensor([ 9,  0,  9,  4,  9,  7,  9, 19,  9,  6,  9, 16,  9,  3,  9, 21], dtype=torch.int32)),
 (tensor([21,  9, 22,  9, 17,  9, 23,  9,  5,  9, 24,  9, 15,  9, 25,  9], dtype=torch.int32),
  tensor([ 9, 22,  9, 17,  9, 23,  9,  5,  9, 24,  9, 15,  9, 25,  9,  1], dtype=torch.int32))]

In [45]:
# want data in order
# (0, m, 2m, ..., )

def group_chunks(dset, bs):
  m = len(dset) // bs
  new_dset = []

  for i in range(m):
    new_dset += [dset[i + m*j] for j in range(bs)]
  return new_dset

In [46]:
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

### Experiment with RNNs
- Write RNNs from scratch

In [53]:
class LMModel1(Module):
  def __init__(self, vocab_sz, n_hidden):
    self.i_h = nn.Embedding(vocab_sz, n_hidden)
    self.h_h = nn.Linear(n_hidden, n_hidden)
    self.h_o = nn.Linear(n_hidden, vocab_sz)
    self.h = 0
  
  def forward(self, x):
    print(x.shape)
    for i in range(x.shape[1]):
      # add the hidden state to the input to hidden state
      self.h += self.i_h(x[:, i])
      # compute new hidden state
      self.h = F.relu(self.h_h(self.h))
    
    # pass the final hidden through the output layer and return
    out = self.h_o(self.h)
    self.h = self.h.detach()

    print(out.shape)

    return out
  
  def reset(self):
    self.h = 0

In [54]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


torch.Size([64, 16])
torch.Size([64, 30])


RuntimeError: ignored